In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maram\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from nltk.corpus import stopwords
stopwords = stopwords.words("english")


In [13]:
# importing the Data
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()


['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [14]:
data = df.content.values.tolist()

In [19]:
# Removing the e-mails and newline characters

data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]
pprint(data[:1])

['From: (wheres my thing)\n'
 'Subject: WHAT car is this!?\n'
 'Nntp-Posting-Host: rac3.wam.umd.edu\n'
 'Organization: University of Maryland, College Park\n'
 'Lines: 15\n'
 '\n'
 ' I was wondering if anyone out there could enlighten me on this car I saw\n'
 'the other day. It was a 2-door sports car, looked to be from the late 60s/\n'
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition,\n'
 'the front bumper was separate from the rest of the body. This is \n'
 'all I know. If anyone can tellme a model name, engine specs, years\n'
 'of production, where this car is made, history, or whatever info you\n'
 'have on this funky looking car, please e-mail.\n'
 '\n'
 'Thanks,\n'
 '- IL\n'
 '   ---- brought to you by your neighborhood Lerxst ----\n'
 '\n'
 '\n'
 '\n'
 '\n']


In [24]:
# Looks data messy, so removing punctuations and unnecesasry characters

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True))


data_words = list(sent_to_words(data))
print(data_words[:1])


[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [51]:
# building the bigram and trigram

bigram = gensim.models.Phrases(data_words,min_count=5,threshold=100)
trigram = gensim.models.Phrases(bigram[data_words],threshold=100)


In [52]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [53]:
print(bigram_mod[data_words[1]])

['from', 'guy_kuo', 'subject', 'si', 'clock', 'poll', 'final', 'call', 'summary', 'final', 'call', 'for', 'si', 'clock', 'reports', 'keywords', 'si', 'acceleration', 'clock', 'upgrade', 'article', 'shelley', 'qvfo', 'innc', 'organization', 'university', 'of', 'washington', 'lines', 'nntp_posting', 'host', 'carson_washington', 'edu', 'fair', 'number', 'of', 'brave', 'souls', 'who', 'upgraded', 'their', 'si', 'clock', 'oscillator', 'have', 'shared', 'their', 'experiences', 'for', 'this', 'poll', 'please', 'send', 'brief', 'message', 'detailing', 'your', 'experiences', 'with', 'the', 'procedure', 'top', 'speed', 'attained', 'cpu', 'rated', 'speed', 'add', 'on', 'cards', 'and', 'adapters', 'heat_sinks', 'hour', 'of', 'usage', 'per', 'day', 'floppy_disk', 'functionality', 'with', 'and', 'floppies', 'are', 'especially', 'requested', 'will', 'be', 'summarizing', 'in', 'the', 'next', 'two', 'days', 'so', 'please', 'add', 'to', 'the', 'network', 'knowledge', 'base', 'if', 'you', 'have', 'done',

In [65]:
# Defined functions

def remove_stopwords(texts):
    return[[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

def make_bigrams(text):
    return[bigram_mod[doc] for doc in text]

def make_trigram(text):
    return[trigram_mod[bigram_mod[doc]] for doc in text]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
    



In [78]:
# Lets call the functions in order

data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)



In [84]:
import en_core_web_sm
nlp = en_core_web_sm.load()


In [85]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [87]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts] # TF-IDF

print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)]]


In [88]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [89]:
pprint(lda_model.print_topics()) # printing the topics
doc_lda = lda_model[corpus]

[(0,
  '0.169*"file" + 0.088*"image" + 0.073*"entry" + 0.058*"program" + '
  '0.045*"server" + 0.039*"format" + 0.026*"announcement" + 0.025*"text" + '
  '0.021*"session" + 0.018*"militia"'),
 (1,
  '0.101*"use" + 0.040*"system" + 0.024*"include" + 0.021*"also" + '
  '0.020*"provide" + 0.018*"source" + 0.016*"available" + 0.014*"type" + '
  '0.014*"require" + 0.013*"datum"'),
 (2,
  '0.043*"run" + 0.035*"card" + 0.032*"window" + 0.032*"problem" + '
  '0.027*"software" + 0.027*"program" + 0.026*"driver" + 0.025*"use" + '
  '0.024*"bit" + 0.022*"set"'),
 (3,
  '0.114*"line" + 0.041*"host" + 0.040*"subject" + 0.036*"organization" + '
  '0.032*"thank" + 0.024*"mail" + 0.022*"help" + 0.022*"post" + 0.021*"send" + '
  '0.020*"new"'),
 (4,
  '0.055*"mouse" + 0.054*"period" + 0.042*"originator" + 0.042*"baseball" + '
  '0.037*"motif" + 0.027*"yesterday" + 0.027*"cool" + 0.025*"rock" + '
  '0.023*"default" + 0.021*"pitch"'),
 (5,
  '0.039*"people" + 0.029*"say" + 0.023*"believe" + 0.023*"reason

In [91]:
# compute the Model Perplexity and model coherence score to judge how good a given topic model is

print('\nPerplexity : ',lda_model.log_perplexity(corpus)) # low score is better

coherence_model_lda = CoherenceModel(model=lda_model,texts=data_lemmatized,dictionary=id2word,coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score : ',coherence_lda)


Perplexity :  -13.528442258344818

Coherence Score :  0.4567774625644919


In [92]:
pyLDAvis.enable_notebook()  # To visualize the topics
vis = pyLDAvis.gensim.prepare(lda_model,corpus,id2word)
vis

C:\Users\maram\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.334280 -0.083712       1        1  41.880318
5     -0.294952 -0.192017       2        1  11.928931
1     -0.215005  0.202602       3        1   9.155848
3     -0.233364 -0.020883       4        1   8.653131
14    -0.143030 -0.203897       5        1   4.964920
2     -0.114796  0.295926       6        1   4.745868
9     -0.038670  0.041773       7        1   3.400816
10    -0.044788  0.065779       8        1   3.359077
13     0.015557  0.099552       9        1   2.221569
6      0.029750 -0.046474      10        1   1.978938
0      0.091966  0.051855      11        1   1.240341
8      0.084520  0.018313      12        1   1.167072
19     0.042168  0.039479      13        1   1.162873
15     0.135198 -0.035644      14        1   0.918545
17     0.137893 -0.033184      15        1   0.844212
7      0.156862 -0.040600      16        1   0.681298
4      0.166928 -0.040793      17        1   0.646873
16     0.170568 -0.031838      18        1   0.424758
12     0.194671 -0.042808      19        1   0.356833
18     0.192804 -0.043430      20        1   0.267796, topic_info=      Category          Freq        Term         Total  loglift  logprob
17     Default  13465.000000        line  13465.000000   30.000  30.0000
162    Default  12453.000000         use  12453.000000   29.000  29.0000
35     Default   4444.000000        year   4444.000000   28.000  28.0000
131    Default   7505.000000      people   7505.000000   27.000  27.0000
30     Default   5407.000000     subject   5407.000000   26.000  26.0000
...        ...           ...         ...           ...      ...      ...
11294  Topic20      0.036400  hypothesis      1.244201    2.391 -11.3325
3422   Topic20      0.036402     connect      1.244274    2.391 -11.3324
4217   Topic20      0.036402   existence      1.244289    2.391 -11.3324
3493   Topic20      0.036401    internet      1.244301    2.391 -11.3325
4251   Topic20      0.036400       laugh      1.244240    2.391 -11.3325

[854 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2977     13  0.965895   abstract
1235      2  0.998836     accept
77        6  0.774378     access
77       13  0.224313     access
1031     20  0.973024   accuracy
...     ...       ...        ...
35        8  0.946946       year
730      17  0.991902  yesterday
563       2  0.001395      young
563       7  0.920734      young
563      10  0.076728      young

[1056 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 6, 2, 4, 15, 3, 10, 11, 14, 7, 1, 9, 20, 16, 18, 8, 5, 17, 13, 19])